In [1]:
# Hente ukes verdier for Teknisk (AUT, DKS og EL) Merit rapport
# *Feilmelinger
# *Aktive AO Teknisk
# *Dagrapport + AO Teknsik
# 
#
#
# SQL
# Komentarer
#                Hel linje          --Hei:
#                Del av linje linje --Hei;
#                På flere linjer    /*Hei
#                                   på deg,*/
# Skrive kode i SQL: 
#                   Start alltid""" og avslutt med""": """Kode"""
#                   Skal man ha Python kode inni en SQL streng må men avslutte og starte, EKS:
#                   SELECT COUNT(TID_D.ANL_NR) as Antall, """+str(weekNumber)+""" as Ukenummer

In [23]:
# Import av alle bibloteker som er brukt for koding
import pandas as pd
import pyodbc
#Importere Dato
from datetime import date
from datetime import datetime, timedelta
import calendar
import sqlalchemy as sa
import urllib
import numpy as np

# Parameters
server = 'veas-sql1.vestfjorden.no'
db = 'Merit_prod'

# Create the connection
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + db + ';Trusted_Connection=yes')


In [3]:
# Legge til ny rad i df med tatalsum for valgt colonne
def sum_radDF(df,col):
    
    # Legge til Total på siste rad
    sum_row = df[[col]].sum()
    # Flip Colonne og Rad
    df_sum_row = pd.DataFrame(data = sum_row).T
    
    df_sum_row = df_sum_row.reindex(columns = df.columns)
    df_out = df.append(df_sum_row, ignore_index=True)
    
    return df_out


In [4]:
# Sette sammen df via felles colonne
def df_join(df1, col1, df2, col2, df3, col3, df4, col4):
    
    #Sette sammen flere df
    df_join_out = df1.join(df2.set_index(col2), on=col1).join(df3.set_index(col3), on=col1).join(df4.set_index(col4), on=col1)
    
    return df_join_out

In [25]:
# Trim DF for alle mellomrom foran og bak tekst
def trimAllColumns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trimStrings = lambda x: x.strip() if type(x) is str else x
    return df.applymap(trimStrings)


# simple example of trimming whitespace from data elements
#df = pd.DataFrame([['  a  ', 10], ['  c  ', 5]])
#df = trimAllColumns(df)
#print(df)

In [6]:
#Lage liste med bestemt bokstav foran (defoult 'E')
def Jonas_AOkode(df_AOliste,Char="E"):
    AOKode =[]
    for i in range(len(df_AOliste)):

        #print('{}{}'.format(chr(69),i))
        AOKode.append("{}{}".format(Char,i+2))
    return AOKode

In [7]:
#Hente ut alle Colonner i en bestemt tabell
q_ColonnerType = """
SELECT TOP 5 *

    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE 
     TABLE_NAME = 'TID_D'
     
"""
#ORDER BY ORDINAL_POSITION
# Hente fram sql
df_ColonnerType = pd.read_sql(q_ColonnerType, conn)
df_ColonnerType

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,...,DATETIME_PRECISION,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME
0,Merit_prod,dbo,TID_D,DATUM,1,None,NO,char,6,6,...,None,None,None,iso_1,None,None,Danish_Norwegian_CI_AS,None,None,None
1,Merit_prod,dbo,TID_D,AO_NR,2,None,NO,char,10,10,...,None,None,None,iso_1,None,None,Danish_Norwegian_CI_AS,None,None,None
2,Merit_prod,dbo,TID_D,ANL_NR,3,None,NO,varchar,50,50,...,None,None,None,iso_1,None,None,Danish_Norwegian_CI_AS,None,None,None
3,Merit_prod,dbo,TID_D,ANL_FAS,4,None,NO,char,2,2,...,None,None,None,iso_1,None,None,Danish_Norwegian_CI_AS,None,None,None
4,Merit_prod,dbo,TID_D,TEXTNYCKEL,5,None,NO,char,12,12,...,None,None,None,iso_1,None,None,Danish_Norwegian_CI_AS,None,None,None


In [8]:
# Hente Tekst fra Dagrapport/AO
q_tekst = '''
SELECT
    Id,
    Text 
    
FROM taText
        WHERE
         Id LIKE 'DR%'
'''
# = RIGHT(iD, LEN(Id) - 2 )
df_tekst = pd.read_sql(q_tekst, conn)
df_tekst = trimAllColumns(df_tekst)
df_tekst.head()

,Id,Text
0,DR000000000001,1 (Feilmelding-bestilling)\r\nUlyd på...
1,DR000000000002,2 (Feilmelding-bestilling)\r\nTest ul...
2,DR000000000003,Test ulyd fra pumpe
3,DR000000000005,4 (Feilmelding-bestilling)\r\nTest
4,DR000000000006,8 (Feilmelding-bestilling)\r\nTest så...


In [9]:
# Finn dato automatisk eller via søk
def finnDato(_AutoDato = False):
    '''finnDato(True) = Find automatic todays dato
    finnDato(False) = Write manual dato in format YYYY-MM-DD (2018-09-28)
    
    Get todays dato automatic or a written manual dato in datoformat with that weeks Monday, Sunday and Weeknumber
    
    Returns:
    _today            = Today or manual dato in dato format 
    _start            = Monday in _todays week in dato format
    _end              = Sunday in _todays week in dato format
    _weekNumber       = Today or manual datos weeknumber 
    _startFormatert   = Monday in _todays week in format('%y%m%d')
    _endFormatert     = Sunday in _todays week in format('%y%m%d')
    '''
    #            
    if _AutoDato == False:
        # Text for what to input
        _datestr = input("Skriv inn dato (ÅÅÅÅ-MM-DD):")
        # Formating text to dato
        _today = datetime.strptime(_datestr, "%Y-%m-%d").date()
    else:
        # finnDato(True) Getting todays dato automatick
        _today = datetime.now().date()
        

    # Getting the week number from the date in _today
    _weekNumber = _today.isocalendar()[1]    
    # Getting the monday in todays week
    _start = _today - timedelta(days=_today.weekday())
    # Getting the last day (dato) in the week
    _end = _start + timedelta(days=6)

    # Formating _start to output: '%y%m%d' (YYMMDD) 
    _startFormatert = _start.strftime('%y%m%d')
    # Formating _end to output: '%y%m%d' (YYMMDD) 
    _endFormatert = _end.strftime('%y%m%d')
    
    print("Today       : " + str(_today))
    print("Start Monday: " + str(_start))
    print("End   Sunday: " + str(_end))
    print("WeekNum     : " + str(_weekNumber))
    print("Today(formated): " + str(_startFormatert))
    print("End  (formated): " + str(_endFormatert))
    
    return _today, _start, _end, _weekNumber, _startFormatert, _endFormatert



In [10]:
# Automatisk dato
today,start,end, weekNumber, startFormatert, endFormatert = finnDato(True)

Today       : 2018-10-04
Start Monday: 2018-10-01
End   Sunday: 2018-10-07
WeekNum     : 40
Today(formated): 181001
End  (formated): 181007


In [11]:
# Manuell dato
#today,start,end, weekNumber, startFormatert, endFormatert = finnDato(False)

In [12]:
df_datoer = pd.DataFrame({'Generet': [today], 'UkeStart': [start], 'UkeSlutt': [end], 'Ukenummer' : [weekNumber] })
df_datoer =df_datoer[['Generet', 'Ukenummer', 'UkeStart', 'UkeSlutt']]
df_datoer

,Generet,Ukenummer,UkeStart,UkeSlutt
0,2018-10-04,40,2018-10-01,2018-10-07


In [13]:
#Fyll inn manuelt dato
### Legge til parameter for søk fra/til!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#DatoInnput = input('fyll inn dato: format ÅÅMMDD \'180917\'') #Manuelt

#Mandag gjelldene uke
DatoInnput = start.strftime('%y%m%d')



print(DatoInnput)

181001


In [14]:
#Hente ut alle Feilmeldinger Teknisk (Live verdi),
### MÅ FÅ LAGRET I EN TABELL FOR HVER UKE

q_FeilmeldingTeknisk = """
SELECT 
        UAV                  AS Avdeling, 
        COUNT(AO_D.AO_NR)    AS Antall_Feilmeldinger 
    FROM AO_D
        WHERE 
            (AO_D.UAV='AUT' 
                OR 
            AO_D.UAV='EL' 
                OR 
            AO_D.UAV='DKS')
        GROUP BY UAV;
"""

# Hente fram sql
df_FeilmeldingTeknisk = pd.read_sql(q_FeilmeldingTeknisk, conn)
# Legge til Total på siste rad
df_FeilmeldingTeknisk2 = sum_radDF(df_FeilmeldingTeknisk, "Antall_Feilmeldinger")

df_FeilmeldingTeknisk2

,Avdeling,Antall_Feilmeldinger
0,AUT,31
1,DKS,34
2,EL,28
3,NaN,93


In [15]:
#Hente ut alle Aktive AO Teknisk (Live verdi)
q_AktiveAOTeknisk = """
SELECT 
        UAV                   AS Avdeling, 
        COUNT(AO_D.AO_NR)     AS Antall_Aktive_AO
    FROM AO_D
        WHERE 
            (AO_D.UAV='AUT' OR AO_D.UAV='EL' OR AO_D.UAV='DKS')
                AND
            (AO_D.STATUS='20' OR AO_D.STATUS='30')
            GROUP BY UAV;
"""

# Hente fram sql
df_AktiveAOTeknisk = pd.read_sql(q_AktiveAOTeknisk, conn)

# Legge til Total på siste rad
df_AktiveAOTeknisk2 = sum_radDF(df_AktiveAOTeknisk, "Antall_Aktive_AO")
df_AktiveAOTeknisk2

,Avdeling,Antall_Aktive_AO
0,AUT,26
1,DKS,10
2,EL,21
3,NaN,57


In [16]:
#Hente ut alle Aktive AO Teknisk Timer (Live verdi)
q_AktiveAOTekniskTimer = """
SELECT 
      UAV                            AS Avdeling,
      CAST(PLAN_TIM/10 AS int) *  CAST(PLAN_MAN AS int) AS Timer_Totalt
   
    FROM AO_D
    WHERE

        (AO_D.UAV='AUT' OR AO_D.UAV='EL' OR AO_D.UAV='DKS')
            AND
        (AO_D.STATUS='20' OR AO_D.STATUS='30')
            AND
        (AO_D.PLAN_TIM > 0)
    
"""
#    CAST(PLAN_TIM/10 AS int)       AS Timer, 
#      CAST(PLAN_MAN AS int)          AS Mann, 
      
#WHERE AO_D.PLAN_TIM/100 >'0'
#            (AO_D.UAV='AUT' OR AO_D.UAV='EL' OR AO_D.UAV='DKS')
#                AND
#            (AO_D.STATUS='20' OR AO_D.STATUS='30')
#GROUP BY UAV;

# Hente fram sql
df_AktiveAOTekniskTimer = pd.read_sql(q_AktiveAOTekniskTimer, conn)
#df_AktiveAOTekniskTimer

#Grupere etter 'Avdeling' og sumere 'Timer_Totalt'. ''reset_index() setter ''Avdeling' og 'Timer_Totalt' på samme index'
df_AktiveAOTekniskTimer2 =df_AktiveAOTekniskTimer.groupby(['Avdeling']).sum().reset_index()
#df_AktiveAOTekniskTimer2

# Legge til Total på siste rad
df_AktiveAOTekniskTimer3 = sum_radDF(df_AktiveAOTekniskTimer2, "Timer_Totalt")
df_AktiveAOTekniskTimer3

,Avdeling,Timer_Totalt
0,AUT,16
1,DKS,19
2,EL,175
3,NaN,210


In [17]:
#Hente ut alle utførste AO/Dagrapport + Totalt Teknisk på siste rad
q_UtførteAOTeknisk = """
SELECT 
        UAV_TILL                AS Avdeling, 
        COUNT(TID_D.ANL_NR)     AS Antall_Utførte 
    FROM TID_D
    
        LEFT JOIN HIST_AO_D on TID_D.AO_NR=HIST_AO_D.AO_NR
            WHERE 
                DATUM >= """+ DatoInnput +"""
                    AND
                    (TID_D.UAV_TILL='AUT' OR TID_D.UAV_TILL='EL' OR TID_D.UAV_TILL='DKS')
                    AND
                (TID_D.AO_NR <'1'
                    OR
                (TID_D.AO_NR >'1' AND HIST_AO_D.STATUS = '90')) 
            GROUP BY UAV_TILL;
"""
##(TID_D.UAV_TILL='AUT' OR TID_D.UAV_TILL='EL' OR TID_D.UAV_TILL='DKS')
#                    AND

#Dato mellom                         #WHERE DATUM BETWEEN '180910' AND '180913'
#Dato fra                            #WHERE DATUM >= '180910' 

# Hente fram sql
df_UtførteAOTeknisk= pd.read_sql(q_UtførteAOTeknisk, conn)

# Legge til Total på siste rad
df_UtførteAOTeknisk2 = sum_radDF(df_UtførteAOTeknisk, "Antall_Utførte")
df_UtførteAOTeknisk2


,Avdeling,Antall_Utførte
0,AUT,12
1,DKS,2
2,EL,11
3,NaN,25


In [18]:
#Alle linjer for Dagrapport/AO
#TID_D.AO_NR, Id = RTRIM(Id)
       
q_AltFraDagrapportOgAO = """

SELECT
        DATUM               AS DATO_Dagrapport, 
        BRUKSDATUM          AS DATOTID_Dagrapport, 
        TID_D.UAV_TILL      AS Avdeling_Til, 
        TID_D.AO_NR         AS AO_NR,
        BEHANDLAD           AS AO_Type, 
        OVERF_UF            AS Dagrapport, 
        TID_D.ANL_NR        AS TAG, 
        ANST_NR             AS Sign_Bestiller, 
        TID_D.AVDELNING     AS Område, 
        TID_D.UAV_FRAN      AS Avdeling_Fra, 
        TID_D.TEXTNYCKEL    AS Beskriving
            
        
    FROM TID_D 

        LEFT JOIN HIST_AO_D on TID_D.AO_NR=HIST_AO_D.AO_NR
            WHERE 
                DATUM >= """+ DatoInnput +"""
                    AND
                (TID_D.UAV_TILL='AUT' OR TID_D.UAV_TILL='EL' OR TID_D.UAV_TILL='DKS')
                    AND 
                (TID_D.AO_NR <'1'
                    OR
                (TID_D.AO_NR >'1' AND HIST_AO_D.STATUS = '90'));
"""

#Dato mellom                         #WHERE DATUM BETWEEN '180910' AND '180913'
#Dato fra                            #WHERE DATUM >= '180910' 


# Hente fram sql
df_AltFraDagrapportOgAO1 = pd.read_sql(q_AltFraDagrapportOgAO, conn)
df_test = df_AltFraDagrapportOgAO1

# Legge til DR for å få lik 'Key' som tekst tabell
df_test['Beskriving'] = 'DR' + df_test['Beskriving'].astype(str)

# Hente fram Tekst for Dagrapport/AO 
df_testText = df_test.join(df_tekst.set_index('Id'), on='Beskriving')
# Fjerne 'Key' tabell fra DF
df_AltFraDagrapportOgAO = df_testText.drop(columns='Beskriving')
df_AltFraDagrapportOgAO = trimAllColumns(df_AltFraDagrapportOgAO)
df_AltFraDagrapportOgAO.head()

,DATO_Dagrapport,DATOTID_Dagrapport,Avdeling_Til,AO_NR,AO_Type,Dagrapport,TAG,Sign_Bestiller,Område,Avdeling_Fra,Text
0,181002,2018-10-02 13:44:00,EL,,,direkt,SED6,rb,SED6,EL,Lagt ut hovedkurs og UPS-mating til SED6 forde...
1,181002,2018-10-02 14:10:00,AUT,,,direkt,RÅT-RÅT-PV55,thfi,RÅT,AUT,Ventil mistet stengt signal hyppig. Viste seg ...
2,181002,2018-10-02 14:18:00,AUT,,,direkt,RIS-AVN-KSV03,krwe,RIS,AUT,"Ventil gikk i timeout, fikk ikke tilbakemeldin..."
3,181002,2018-10-02 14:19:00,AUT,,,direkt,AVV-PR4-KSV04,krwe,AVV,AUT,Ventil lakk luft. fant ut at det var aktuatore...
4,181002,2018-10-02 14:16:00,AUT,,,direkt,PHA3-SED3-QI11,thfi,PHA3,AUT,Analysator hadde feil på standardløsning 1 og ...


In [19]:
#Sette sammen Feilmalding, Aktive og Historiske AO/Dagrapport til en dataframe med 'Avdeling' som tilknyttning
df_Teknisk = df_join(df_FeilmeldingTeknisk2,    'Avdeling', 
                     df_AktiveAOTeknisk2,       'Avdeling', 
                     df_AktiveAOTekniskTimer3,  'Avdeling',
                     df_UtførteAOTeknisk2,      'Avdeling')

## df_UtførteAOTeknisk2,      'Avdeling'
###df_AktiveAOTekniskTimer3,  'Avdeling',

#Sette inn 'TEKNISK' i ny colonne for totalt i Calum 'Avdeling'
df_Teknisk.iloc[3, df_Teknisk.columns.get_loc('Avdeling')] ='TEKNISK'
df_Teknisk

#df_FeilmeldingTeknisk2 df_AktiveAOTeknisk2 df_UtførteAOTeknisk2

,Avdeling,Antall_Feilmeldinger,Antall_Aktive_AO,Timer_Totalt,Antall_Utførte
0,AUT,31,26,16,12
1,DKS,34,10,19,2
2,EL,28,21,175,11
3,TEKNISK,93,57,210,25


In [20]:
##############################################################################
#Lagre til Excel uten hyperlink

# A simple example of converting a Pandas dataframe to an xlsx file using
# Pandas and XlsxWriter.
#
# Copyright 2013-2018, John McNamara, jmcnamara@cpan.org
## https://github.com/jmcnamara/XlsxWriter/blob/master/dev/docs/source/working_with_pandas.rst

# Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter('UkeRapportMerit'+str(weekNumber)+'.xlsx', engine='xlsxwriter')

# Position the dataframes in the worksheet.
#df_datoer.to_excel(writer, sheet_name='Uke'+str(weekNumber))  # Default position, cell A1.
#df_Teknisk.to_excel(writer, sheet_name='Uke'+str(weekNumber), startrow=3) 

# Convert the dataframe to an XlsxWriter Excel object.
#df_AltFraDagrapportOgAO.to_excel(writer, sheet_name='Dagrapport_Alt')


# Get the xlsxwriter objects from the dataframe writer object.
#workbook  = writer.book
#worksheet1 = writer.sheets['Uke'+str(weekNumber)]
#worksheet1.write_url('A1', 'http://merit.veas.nu/Merit/index.html#Start', string='Merit')
#hyperlink Merit AO http://merit.veas.nu/Merit/index.html#Arbetsorder/Registrera/Arbetsorder?AoNrAo= <---##### sett inn AO nr

# Close the Pandas Excel writer and output the Excel file.
#writer.save()

---

In [21]:
# Kun brukt i Exceldokument, hyperlink for AO-nummer
# Henter fram indeks for AO_NR som kommer fram fra dagrapportsøket
df_AOliste = df_AltFraDagrapportOgAO['AO_NR']

# Bruker listen for AO plasering og lager plasering for Exceldokument hvor hyperlink skal plaseres
test = Jonas_AOkode(df_AOliste)
for i in test:
    print(i)
#Liste med lengde
#a = range(1,df_AOliste.count(), 1)
#print("{}\t{}".format(chr(69),a))

E2
E3
E4
E5
E6
E7
E8
E9
E10
E11
E12
E13
E14
E15
E16
E17
E18
E19
E20
E21
E22
E23
E24
E25
E26


In [24]:
##############################################################################
#
# A simple example of converting a Pandas dataframe to an xlsx file using
# Pandas and XlsxWriter.
#
# Copyright 2013-2018, John McNamara, jmcnamara@cpan.org
## https://github.com/jmcnamara/XlsxWriter/blob/master/dev/docs/source/working_with_pandas.rst

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('UkeRapportMerit'+str(weekNumber)+'.xlsx', engine='xlsxwriter')

# Position the dataframes in the worksheet.
df_datoer.to_excel(writer, sheet_name='Uke'+str(weekNumber))  # Default position, cell A1.
df_Teknisk.to_excel(writer, sheet_name='Uke'+str(weekNumber), startrow=3) 

# Convert the dataframe to an XlsxWriter Excel object.
df_AltFraDagrapportOgAO.to_excel(writer, sheet_name='Dagrapport_Alt') 
#df_ColonnerType.to_excel(writer, sheet_name='Tabell') 

# Get the xlsxwriter objects from the dataframe writer object.
## Lage hyperlink for tag og
workbook  = writer.book
worksheet1 = writer.sheets['Dagrapport_Alt']

# Liste med plasering for hyperlink
liste = Jonas_AOkode(df_AOliste)
# url hyperlink for AO_NR direkte til merit.nu med blankt for selve AO
url = 'http://merit.veas.nu/Merit/index.html#Arbetsorder/Registrera/Arbetsorder?AoNrAo='
# Link for AO_NR som blir brukt i slutten av hyperlinken
link = df_AOliste
index = 0
for i in liste:
    url2 = url + link[index]
    worksheet1.write_url(i, url2, string=link[index])
    index += 1

# Close the Pandas Excel writer and output the Excel file.
writer.save()